# meerkat yale Fossils sorting sandbox

Created on: Thursday Aug 4th, 2022  
Created by: Jacob A Rose

In [1]:
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_colwidth', 1000)



from typing import *
from rich import print as pp
import os
import numpy as np
import inspect
from tqdm.auto import tqdm
from pathlib import Path
import logging
# from imutils.catalog_registry import available_datasets

from PIL.ImageStat import Stat
import meerkat as mk

In [ ]:
display_res = 512
# print(mk.config.DisplayOptions.max_image_width)
mk.config.DisplayOptions.max_image_width = display_res
mk.config.DisplayOptions.max_image_height = display_res
print(f"{mk.config.DisplayOptions.max_image_width=}")

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import fiftyone as fo
import fiftyone.zoo as foz
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



dataset = foz.load_zoo_dataset("cifar100", shuffle=True, max_samples=1000)
model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")

embeddings = dataset.compute_embeddings(model)
print(embeddings.shape)



similarity_matrix = cosine_similarity(embeddings)
print(similarity_matrix.shape)
print(similarity_matrix)




n = len(similarity_matrix)
similarity_matrix = similarity_matrix - np.identity(n)

from tqdm.auto import tqdm

id_map = [s.id for s in dataset.select_fields(["id"])]

for idx, sample in enumerate(tqdm(dataset)):
    sample["max_similarity"] = similarity_matrix[idx].max()
    sample.save()

session = fo.launch_app(dataset, remote=True)#, port="9898")

In [2]:
from typing import *
from pathlib import Path

def extract_dataset_from_file_list(fpaths: List):
    return {
        "paths" : fpaths,
        "file_ids" : [Path(p).stem for p in fpaths],
        "imgs" : [imread(p) for p in fpaths]
    }

    
def extract_dataset_from_directory(parent_dir, max_imgs: Optional[int]=None):
    
    blacklist = [".ipynb_checkpoints"]
    fpaths = sorted([os.path.join(parent_dir, p) for p in os.listdir(parent_dir) if p not in blacklist])
    
    if isinstance(max_imgs, int):
        fpaths = fpaths[:max_imgs]
    
    return extract_dataset_from_file_list(fpaths)

In [3]:
yale_fossil_dir = "/media/data_cifs/projects/prj_fossils/data/yale_full"

In [4]:
img_dict = extract_dataset_from_directory(parent_dir=yale_fossil_dir, max_imgs=5)

In [6]:
img_dict['imgs'][1]

dask.array<_map_read_frame, shape=(1, 6000, 3728, 3), dtype=uint8, chunksize=(1, 6000, 3728, 3), chunktype=numpy.ndarray>

In [ ]:
import fiftyone as fo

# Create a dataset from a list of images
dataset = fo.Dataset.from_images(
    ["/path/to/image1.jpg", "/path/to/image2.jpg", ...]
)

# Create a dataset from a directory of images
dataset = fo.Dataset.from_images_dir("/path/to/images")

# Create a dataset from a glob pattern of images
dataset = fo.Dataset.from_images_patt("/path/to/images/*.jpg")

session = fo.launch_app(dataset)

In [1]:
# %gui qt5
# from skimage import data
# from napari import ViewerApp
# viewer = ViewerApp(data.astronaut())

ImportError: cannot import name 'ViewerApp' from 'napari' (/media/data/conda/jrose3/envs/napari-env/lib/python3.9/site-packages/napari/__init__.py)

In [1]:
import qtpy
# dir(qtpy)

In [2]:
qtpy.__version__

'2.1.0'

In [3]:
qtpy.QT_VERSION

'5.9.7'

In [4]:
print(get_ipython().active_eventloop)

None


In [ ]:
v = napari.view_image(
    img_dict['imgs'],
    # contrast_limits=[90, 1500],
    multiscale=False,
    ndisplay=3,
    # scale=(3, 1, 1),
)

napari.run()

/media/data/conda/jrose3/envs/napari-env/lib/python3.9/site-packages/napari/_qt/__init__.py:53: UserWarning: 

napari was tested with QT library `>=5.12.3`.
The version installed is 5.9.7. Please report any issues with
this specific QT version at https://github.com/Napari/napari/issues.
  warn(message=warn_message)
